In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Autoencoder(nn.Module):
    def __init__(self, input_size, hidden_size, latent_size, num_layers):
        super(Autoencoder, self).__init__()

        self.encoder = nn.Sequential()
        self.decoder = nn.Sequential()
        
        for i in range(num_layers):
            if i == 0:
                self.encoder.add_module('encoder_layer{}'.format(i), nn.Linear(input_size, hidden_size))
                self.encoder.add_module('encoder_relu{}'.format(i), nn.ReLU())
                self.decoder.add_module('decoder_layer{}'.format(i), nn.Linear(latent_size, hidden_size))
                self.decoder.add_module('decoder_relu{}'.format(i), nn.ReLU())
            elif i == num_layers - 1:
                self.encoder.add_module('encoder_layer{}'.format(i), nn.Linear(hidden_size, latent_size))
                self.decoder.add_module('decoder_layer{}'.format(i), nn.Linear(hidden_size, input_size))
            else:
                self.encoder.add_module('encoder_layer{}'.format(i), nn.Linear(hidden_size, hidden_size))
                self.decoder.add_module('decoder_layer{}'.format(i), nn.Linear(hidden_size, hidden_size))
                self.encoder.add_module('encoder_relu{}'.format(i), nn.ReLU())
                self.decoder.add_module('decoder_relu{}'.format(i), nn.ReLU())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
class VAE(nn.Module):
    def __init__(self, input_size, hidden_size, latent_size, num_layers):
        super(VAE, self).__init__()

        self.encoder = nn.Sequential()
        self.decoder = nn.Sequential()

        for i in range(num_layers):
            if i == 0:
                self.encoder.add_module('encoder_layer{}'.format(i), nn.Linear(input_size, hidden_size))
                self.encoder.add_module('encoder_relu{}'.format(i), nn.ReLU())
                self.decoder.add_module('decoder_layer{}'.format(i), nn.Linear(latent_size, hidden_size))
                self.decoder.add_module('decoder_relu{}'.format(i), nn.ReLU())
            elif i == num_layers - 1:
                self.encoder.add_module('encoder_layer{}'.format(i), nn.Linear(hidden_size, latent_size))
                self.decoder.add_module('decoder_layer{}'.format(i), nn.Linear(hidden_size, input_size))
            else:
                self.encoder.add_module('encoder_layer{}'.format(i), nn.Linear(hidden_size, hidden_size))
                self.decoder.add_module('decoder_layer{}'.format(i), nn.Linear(hidden_size, hidden_size))
                self.encoder.add_module('encoder_relu{}'.format(i), nn.ReLU())
                self.decoder.add_module('decoder_relu{}'.format(i), nn.ReLU())
        
        self.mu = nn.Linear(latent_size, latent_size)
        self.logvar = nn.Linear(latent_size, latent_size)

    def forward(self, x):
        x = self.encoder(x)
        mu = self.mu(x)
        logvar = self.logvar(x)
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        x = eps.mul(std).add_(mu)
        x = self.decoder(x)
        return x, mu, logvar
    


In [ ]:


X = torch.randn(100, 784)
model = Autoencoder(input_size=784, hidden_size=400, latent_size=20)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
for epoch in range(100):
    X_hat = model(X)
    loss = F.mse_loss(X_hat, X)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

X = torch.randn(100, 784)
model = VAE(input_size=784, hidden_size=400, latent_size=20)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
for epoch in range(100):
    X_hat, mu, logvar = model(X)
    loss = F.mse_loss(X_hat, X) + 0.5 * torch.sum(torch.exp(logvar) + mu**2 - 1. - logvar)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
X = torch.randn(100, 784)
Y = torch.randn(100, 784)
model = GAN(input_size=784, hidden_size=400, latent_size=20)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
for epoch in range(100):
    X_hat, mu, logvar, D = model(X)

In [ ]:
#GAN model for 1D time series data, with the goal of predicting the next time step

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

# Define the model
class GAN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GAN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.input_size = input_size
        
        self.encoder = nn.Sequential(
            nn.Linear(self.input_size, self.hidden_size),
            nn.ReLU(),
            nn.Linear(self.hidden_size, self.output_size),
            nn.ReLU()
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(self.output_size, self.hidden_size),
            nn.ReLU(),
            nn.Linear(self.hidden_size, self.input_size),
            nn.ReLU()
        )
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
    def predict(self, x):
        x = self.encoder(x)
        return x
    
    def loss(self, x, y):
        x = self.forward(x)
        return F.mse_loss(x, y)
    
    def train(self, x, y, epochs=1000, lr=0.01):
        optimizer = optim.Adam(self.parameters(), lr=lr)
        for epoch in range(epochs):
            optimizer.zero_grad()
            loss = self.loss(x, y)
            loss.backward()
            optimizer.step()
            if epoch % 100 == 0:
                print('Epoch: {} Loss: {}'.format(epoch, loss.item()))

In [10]:
import torch    
import torch.nn as nn
import torch.nn.functional as F

X = torch.randn(64, 100) # 64 samples, 100 features
y = torch.randn(64, 100) # 64 samples, 1 target
train_dataset = torch.utils.data.TensorDataset(X, y)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

model = Autoencoder(input_size=100, hidden_size=50)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
for epoch in range(10):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.mse_loss(output, target)
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))

def loss_function(output, target, mu, logvar):
    BCE = F.mse_loss(output, target)

    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD     
model = VAE(input_size=100, hidden_size=50, latent_size=10)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
for epoch in range(10):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output, mu, logvar = model(data)

        loss = loss_function(output, target, mu, logvar)
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))


Train Epoch: 0 [0/64 (0%)]	Loss: 1.029464
Train Epoch: 1 [0/64 (0%)]	Loss: 1.021600
Train Epoch: 2 [0/64 (0%)]	Loss: 1.014361
Train Epoch: 3 [0/64 (0%)]	Loss: 1.007731
Train Epoch: 4 [0/64 (0%)]	Loss: 1.001698
Train Epoch: 5 [0/64 (0%)]	Loss: 0.996092
Train Epoch: 6 [0/64 (0%)]	Loss: 0.990910
Train Epoch: 7 [0/64 (0%)]	Loss: 0.986023
Train Epoch: 8 [0/64 (0%)]	Loss: 0.981444
Train Epoch: 9 [0/64 (0%)]	Loss: 0.977138
Train Epoch: 0 [0/64 (0%)]	Loss: 32.458233
Train Epoch: 1 [0/64 (0%)]	Loss: 28.404478
Train Epoch: 2 [0/64 (0%)]	Loss: 24.836149
Train Epoch: 3 [0/64 (0%)]	Loss: 21.695860
Train Epoch: 4 [0/64 (0%)]	Loss: 18.963074
Train Epoch: 5 [0/64 (0%)]	Loss: 16.590847
Train Epoch: 6 [0/64 (0%)]	Loss: 14.542610
Train Epoch: 7 [0/64 (0%)]	Loss: 12.766900
Train Epoch: 8 [0/64 (0%)]	Loss: 11.267582
Train Epoch: 9 [0/64 (0%)]	Loss: 9.981268


In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x